# Search Images by other Images

In [ ]:
import json

from os import makedirs, path
from PIL import Image as PImage

## Crop Objects from Images

In [ ]:
DATA_FILE = "./metadata/json/20250422_full.json"
IMGS_PATH = "../../imgs/arts/full"
CROP_IMGS_PATH = "../../imgs/arts/crops"

makedirs(CROP_IMGS_PATH, exist_ok=True)

In [ ]:
with open(DATA_FILE, "r", encoding="utf8") as f:
  m_data = json.load(f)

In [ ]:
for qid,data in m_data.items():
  img_path = path.join(IMGS_PATH, f"{qid}.jpg")
  if (not path.isfile(img_path)) or len(data["objects"]) < 1:
    continue

  img = PImage.open(img_path)
  iw,ih = img.size

  for idx,obj in enumerate(data["objects"]):
    x0,y0,x1,y1 = obj["box"]
    label = obj["label"]
    img_c = img.crop((int(x0*iw), int(y0*ih), int(x1*iw), int(y1*ih)))
    idx_str = f"00000{idx}"[-4:]
    img_c.save(path.join(CROP_IMGS_PATH, f"{qid}_{idx_str}.jpg"))

## Calculate Crop Embeddings

In [ ]:
import json

from os import listdir, makedirs, path
from PIL import Image as PImage

from models.CLIP_embedding import Clip
from models.Owlv2 import Owlv2
from models.SigLip2 import SigLip2

In [ ]:
CROP_IMGS_PATH = "../../imgs/arts/crops"
CROP_EMBED_PATH = "./metadata/json/crops/embeddings"

makedirs(CROP_EMBED_PATH, exist_ok=True)

In [ ]:
model = SigLip2()
model_name = type(model).__name__.lower()

In [ ]:
crop_fnames = sorted([fn for fn in listdir(CROP_IMGS_PATH) if fn.endswith(".jpg")])

In [ ]:
for idx,fname in enumerate(crop_fnames):
  qid = fname.replace(".jpg", "")
  image_path = path.join(CROP_IMGS_PATH, fname)
  embedding_path = path.join(CROP_EMBED_PATH, f"{qid}.json")

  embeds = {}
  if path.isfile(embedding_path):
    with open(embedding_path, "r") as ifp:
      embeds = json.load(ifp)[qid]

  if model_name in embeds:
    continue

  img = PImage.open(image_path)
  embeds[model_name] = [round(v, 8) for v in model.get_embedding(img).tolist()]
  embedding_data = { qid: embeds }

  with open(embedding_path, "w") as ofp:
    json.dump(embedding_data, ofp, separators=(",",":"), sort_keys=True, ensure_ascii=False)

## Search by Image

In [ ]:
import json

from os import listdir, makedirs, path

from PIL import Image as PImage, ImageOps as PImageOps, ImageDraw as PImageDraw


### Open Tree image

## SigLip2 (embedding)

Large patch 16x16:
- https://huggingface.co/google/siglip2-large-patch16-384
- https://huggingface.co/google/siglip2-large-patch16-512

Giant 16x16:
- https://huggingface.co/google/siglip2-giant-opt-patch16-256
- https://huggingface.co/google/siglip2-giant-opt-patch16-384


## Owl2 (zero-shot detection)
- https://huggingface.co/google/owlv2-base-patch16
- https://huggingface.co/google/owlv2-large-patch14
- https://huggingface.co/google/owlv2-large-patch14-ensemble

- [Niels' Tutorial](https://github.com/NielsRogge/Transformers-Tutorials/blob/master/OWLv2/Zero_and_one_shot_object_detection_with_OWLv2.ipynb)

#### Results from experiments
- Use larger images and smaller models